In [41]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
eps = np.finfo(float).eps
import operator
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pprint
import random
import sys
import csv

In [0]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [0]:
# data = pd.read_csv("/content/gdrive/My Drive/SMAI_Assignments_colab/apparel-trainval.csv")

In [0]:
def process_data():
#     data = pd.read_csv(path)
    y = data[['label']].values
    X = data.drop(data.columns[0], axis = 1)
    X_normalized = StandardScaler().fit_transform(X)
    X_train, X_validate, y_train, y_validate = train_test_split(X_normalized, y, test_size = 0.2, random_state=42)
    return X_train, X_validate, y_train, y_validate

In [0]:
# X_train, X_validate, y_train, y_validate = process_data()

In [0]:
class neural_network(object):
    def __init__(self, activation_func, activation_func_derivative, alpha, ILS, OLS, hiddenLayers):
        self.weightMatrix = []
        self.allInputs = []
        self.allOutputs = []
        self.inputLayerSize = ILS
        self.outputLayerSize = OLS
        self.hiddenLayers = hiddenLayers
        self.activation_func = activation_func
        self.activation_func_derivative = activation_func_derivative
        self.alpha = alpha
        
        if(len(self.hiddenLayers) == 0):
            bound = np.sqrt(1./(self.inputLayerSize))
            self.weightMatrix.append(np.random.uniform(-bound, bound, (self.inputLayerSize, self.outputLayerSize)))
        else:
            bound = np.sqrt(1./(self.hiddenLayers[0]))
            self.weightMatrix.append(np.random.uniform(-bound, bound, (self.inputLayerSize, self.hiddenLayers[0])))
            for x in range(1,len(self.hiddenLayers)):
                bound = np.sqrt(1./(self.hiddenLayers[x]))
                self.weightMatrix.append(np.random.uniform(-bound, bound, (self.hiddenLayers[x-1], self.hiddenLayers[x])))
            self.weightMatrix.append(np.random.uniform(-bound, bound, (self.hiddenLayers[len(self.hiddenLayers)-1], self.outputLayerSize)))
    
    def forwardPropagation(self, Input):
        self.allInputs = []
        self.allOutputs = []
        for i in range(0, len(self.weightMatrix)):
            
            if(i == 0):
                self.allInputs.append(np.matmul(Input,self.weightMatrix[0]))
            else:
                self.allInputs.append(np.matmul(self.allOutputs[i-1],self.weightMatrix[i]))
            
            if (i == len(self.weightMatrix)-1):
                self.allOutputs.append(stable_softmax(self.allInputs[i]))
            else:
                self.allOutputs.append(self.activation_func(self.allInputs[i]))
        return self.allOutputs
    
    def backPropagation(self, encoded, input):
        delta_list = [0]*len(self.weightMatrix)
        for i in reversed(range(len(self.weightMatrix))):
            if(i == len(self.weightMatrix) - 1):
                delta = np.multiply((self.allOutputs[i] - encoded), softmax_derivative(self.allInputs[i]))
            else:
                delta = np.multiply(np.dot(delta_list[i+1], self.weightMatrix[i+1].T), self.activation_func_derivative(self.allInputs[i]))
            delta_list[i] = delta
        for i in range(0, len(self.weightMatrix)):
            if(i == 0):
                self.weightMatrix[i] -= self.alpha * np.dot(input.T, delta_list[0])
            else:
                self.weightMatrix[i] -= self.alpha * np.dot(self.allOutputs[i-1].T, delta_list[i])
                
    def predictionOnTest(model_test, data):
      outputs = model_test.forwardPropagation(data)
      predictions = np.argmax(outputs[-1], axis=1)
      return predictions

In [0]:
def oneHotEncode(data):
    n_classes = len(np.unique(data))
    targets = np.array(data).reshape(-1)
    return np.eye(n_classes)[targets]

In [0]:
# oneHot = oneHotEncode(y_train)

In [0]:
def crossEntropy(prediction, target, epsilon=1e-12):
    prediction = np.clip(prediction, epsilon, 1. - epsilon)
    N = prediction.shape[0]
    c_entropy = -np.sum(target * np.log(prediction+1e-9))/N
    return c_entropy

In [0]:
def ReLU_derivative(matrix):
    matrix[matrix > 0] = 1
    matrix[matrix <= 0] = 0
    return matrix

In [0]:
 def ReLU(matrix):
        return np.maximum(matrix, 0.0, matrix)

In [0]:
def sigmoid(matrix):
  sigm = 1. / (1. + np.exp(-matrix))
  return sigm

In [0]:
def sigmoid_derivative(matrix):
  sigm = sigmoid(matrix)
  return sigm * (1. - sigm)

In [0]:
def tan_h(matrix):
  return np.tanh(matrix)

In [0]:
def tan_h_derivative(matrix):
  return 1.0 - np.tanh(matrix)**2

In [0]:
def stable_softmax(matrix):
        shift_x = matrix - np.array([np.max(matrix, axis=1)]).T
        exps = np.exp(shift_x)
        return exps / np.array([np.sum(exps, axis=1)]).T

In [0]:
def softmax_derivative(matrix):
    temp = stable_softmax(matrix)
    return temp*(1 - temp)

In [0]:
def trainOnRelu(model_relu, epochs):
    error_list_relu = []
    iter_list = []
    for i in range(epochs):
      print("relu : ", i)
      outputs = model_relu.forwardPropagation(X_train)
      c_entropy = crossEntropy(outputs[-1], oneHot)
      error_list_relu.append(c_entropy)
      iter_list.append(i)
      model_relu.backPropagation(oneHot, X_train)
    return error_list_relu

In [0]:
def trainOnSigmoid(model_sigmoid, epochs):
    error_list_sigmoid = []
    iter_list = []
    for i in range(epochs):
      print("sigmoid : ", i)
      outputs = model_sigmoid.forwardPropagation(X_train)
      c_entropy = crossEntropy(outputs[-1], oneHot)
      error_list_sigmoid.append(c_entropy)
      iter_list.append(i)
      model_sigmoid.backPropagation(oneHot, X_train)
    return error_list_sigmoid

In [0]:
def trainOnTanh(model_tanh, epochs):
    error_list_tanh = []
    iter_list = []
    for i in range(epochs):
      print("tanh : ", i)
      outputs = model_tanh.forwardPropagation(X_train)
      c_entropy = crossEntropy(outputs[-1], oneHot)
      error_list_tanh.append(c_entropy)
      iter_list.append(i)
      model_tanh.backPropagation(oneHot, X_train)
    return error_list_tanh

In [0]:
def miniBatchTrain(miniBatchModel, batchSize, epochs):
    batchCount = int(X_train.shape[0] / batchSize)
    miniBatchError = []
    for iters in range(epochs):
      print("miniBatch : ", iters)
      startIndex = 0
      endIndex = batchSize
      for batchNo in range(batchCount):
        X_miniBatch = X_train[startIndex:endIndex]
        outputs = miniBatchModel.forwardPropagation(X_miniBatch)
        y_miniBatch = oneHot[startIndex:endIndex]
        if(batchNo == batchCount - 1):
          c_entropy = crossEntropy(outputs[-1], y_miniBatch)
          miniBatchError.append(c_entropy)
        miniBatchModel.backPropagation(y_miniBatch, X_miniBatch)
        startIndex = endIndex
        endIndex += batchSize
    return miniBatchError

In [0]:
def plotBatch(error_list_relu, error_list_sigmoid, error_list_tanh, iter_list):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.scatter(iter_list, error_list_relu, color="blue",marker = '.', s= 30, label = 'ReLU')
    ax.scatter(iter_list, error_list_sigmoid, color="black",marker = '.', s= 30, label = 'Sigmoid')
    ax.scatter(iter_list, error_list_tanh, color="red",marker = '.', s= 30, label = 'Tanh')
    plt.legend()
    plt.xlabel("Iterations")
    plt.ylabel("Cross Entropy Error")
    plt.title("Iterations vs Model Cost : Batch Mode")
    plt.show()

In [0]:
def plotMiniBatch(error_list_relu, error_list_sigmoid, error_list_tanh, iter_list):
    fig1, ax1 = plt.subplots(figsize=(12,6))
    ax1.scatter(iter_list, error_list_relu, color="blue",marker = '.', s= 30, label = 'ReLU')
    ax1.scatter(iter_list, error_list_sigmoid, color="black",marker = '.', s= 30, label = 'Sigmoid')
    ax1.scatter(iter_list, error_list_tanh, color="red",marker = '.', s= 30, label = 'Tanh')
    plt.legend()
    plt.xlabel("Iterations")
    plt.ylabel("Cross Entropy Error")
    plt.title("Iterations vs Model Cost : Mini Batch Mode")
    plt.show()

In [0]:
def train(trainType):
    hiddenLayers = [60,60]
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y_train))
    alpha_relu = 0.0001
    alpha_sigmoid = 0.0001
    alpha_tanh = 0.0001
    
    if(trainType == "batch"):
      model_relu = neural_network(ReLU, ReLU_derivative, alpha_relu, input_dim, output_dim, hiddenLayers)
      model_sigmoid = neural_network(sigmoid, sigmoid_derivative, alpha_sigmoid, input_dim, output_dim, hiddenLayers)
      model_tanh = neural_network(tan_h, tan_h_derivative, alpha_tanh, input_dim, output_dim, hiddenLayers)
      epochs = 500
      relu_error = trainOnRelu(model_relu, epochs)
      sigmoid_error = trainOnSigmoid(model_sigmoid, epochs)
      tanh_error = trainOnTanh(model_tanh, epochs)
      iter_list = range(0, epochs)
      plotBatch(relu_error, sigmoid_error, tanh_error, iter_list)
      
      print("Minimum Error by ReLU : ", np.min(relu_error))
      print("Minimum Error by Tanh : ", np.min(tanh_error))
      print("Minimum Error by Sigmoid : ", np.min(sigmoid_error))
      
      
      print("\n\nACCURACY USING BATCH MODE: ")
      # Relu accuracy
      val_output = model_relu.forwardPropagation(X_validate)
      predictions = np.argmax(val_output[-1], axis=1)
      y_val = np.argmax(oneHotEncode(y_validate), axis = 1)
      print("RELU : ", (np.mean(predictions == y_val))*100)

      # Sigmoid accuracy
      val_output = model_sigmoid.forwardPropagation(X_validate)
      predictions = np.argmax(val_output[-1], axis=1)
      y_val = np.argmax(oneHotEncode(y_validate), axis = 1)
      print("SIGMOID : ", (np.mean(predictions == y_val))*100)

      # Tanh accuracy
      val_output = model_tanh.forwardPropagation(X_validate)
      predictions = np.argmax(val_output[-1], axis=1)
      y_val = np.argmax(oneHotEncode(y_validate), axis = 1)
      print("TANH : ", (np.mean(predictions == y_val))*100)
      
      # saving weights
      weight_batch_relu = model_relu.weightMatrix
      np.save('weights_batch_relu', weight_batch_relu)
      
      weight_batch_sigmoid = model_sigmoid.weightMatrix
      np.save('weights_batch_sigmoid', weight_batch_sigmoid)
      
      weight_batch_tanh = model_tanh.weightMatrix
      np.save('weights_batch_tanh', weight_batch_tanh)
      
      
      
    elif(trainType == "miniBatch"):
      iters = 500
      batchSize = 32
      iter_list = range(0, iters)
      model_miniBatch_relu = neural_network(ReLU, ReLU_derivative, alpha_relu, input_dim, output_dim, hiddenLayers)
      model_miniBatch_sigmoid = neural_network(sigmoid, sigmoid_derivative, alpha_sigmoid, input_dim, output_dim, hiddenLayers)
      model_miniBatch_tanh = neural_network(tan_h, tan_h_derivative, alpha_tanh, input_dim, output_dim, hiddenLayers)
#       miniBatchError_relu = miniBatchTrain(model_miniBatch_relu, batchSize, iters)
#       miniBatchError_sigmoid = miniBatchTrain(model_miniBatch_sigmoid, batchSize, iters)
      miniBatchError_tanh = miniBatchTrain(model_miniBatch_tanh, batchSize, iters)
      fig1, ax1 = plt.subplots(figsize=(12,6))
      ax1.scatter(iter_list, miniBatchError_tanh, color="blue",marker = '.', s= 30, label = 'Tanh')
      plt.legend()
      plt.xlabel("Iterations")
      plt.ylabel("Model Loss")
      plt.title("Iterations vs Model Loss : Mini Batch Mode")
      plt.show()
      
      print("\n\nACCURACY USING MINI BATCH: ")
#       val_output = model_miniBatch_relu.forwardPropagation(X_validate)
#       predictions = np.argmax(val_output[-1], axis=1)
#       y_val = np.argmax(oneHotEncode(y_validate), axis = 1)
#       print("RELU : ", (np.mean(predictions == y_val))*100)
      
#       val_output = model_miniBatch_sigmoid.forwardPropagation(X_validate)
#       predictions = np.argmax(val_output[-1], axis=1)
#       y_val = np.argmax(oneHotEncode(y_validate), axis = 1)
#       print("SIGMOID : ", (np.mean(predictions == y_val))*100)
      
      val_output = model_miniBatch_tanh.forwardPropagation(X_validate)
      predictions = np.argmax(val_output[-1], axis=1)
      y_val = np.argmax(oneHotEncode(y_validate), axis = 1)
      print("TANH : ", (np.mean(predictions == y_val))*100)
      
      # saving weights
#       weight_mini_batch_relu = model_miniBatch_relu.weightMatrix
#       np.save('weights_mini_batch_relu', weight_mini_batch_relu)
      
#       weight_mini_batch_sigmoid = model_miniBatch_sigmoid.weightMatrix
#       np.save('weights_mini_batch_sigmoid', weight_mini_batch_sigmoid)
      
      weight_mini_batch_tanh = model_miniBatch_tanh.weightMatrix
      np.save('/content/gdrive/My Drive/SMAI_Assignments_colab/weights_mini_batch_tanh', weight_mini_batch_tanh)

    else:
      print("Wrong Training Method Provided")
      sys.exit()


In [0]:
def test(test_data):
  alpha_tanh = 0.0001
  input_dim = test_data.shape[1]
  output_dim = 10
  hiddenLayers = [60, 60]
  model_test = neural_network(tan_h, tan_h_derivative, alpha_tanh, input_dim, output_dim, hiddenLayers)
  weights = np.load('/content/gdrive/My Drive/SMAI_Assignments_colab/weights_mini_batch_tanh.npy')
  weights = weights.tolist()
  model_test.weightMatrix = weights
  predictions = model_test.predictionOnTest(test_data)
  fileData = [[i] for i in predictions]
  with open('/content/gdrive/My Drive/SMAI_Assignments_colab/2018201057_prediction.csv', 'w') as outputFile:
    writer = csv.writer(outputFile)
    writer.writerows(fileData)
  outputFile.close()

In [0]:
# hiddenLayers = [60,60]
# input_dim = X_train.shape[1]
# output_dim = len(np.unique(y_train))
# alpha_relu = 0.0001
# alpha_sigmoid = 0.0001
# alpha_tanh = 0.0001
# model_relu = neural_network(ReLU, ReLU_derivative, alpha_relu, input_dim, output_dim, hiddenLayers)
# model_sigmoid = neural_network(sigmoid, sigmoid_derivative, alpha_sigmoid, input_dim, output_dim, hiddenLayers)
# model_tanh = neural_network(tan_h, tan_h_derivative, alpha_tanh, input_dim, output_dim, hiddenLayers)

In [0]:
# error_list_relu = []
# iter_list = []
# for i in range(400):
#         outputs = model_relu.forwardPropagation(X_train)
#         c_entropy = crossEntropy(outputs[-1], oneHot)
# #         print(i, c_entropy)
#         error_list_relu.append(c_entropy)
#         iter_list.append(i)
#         model_relu.backPropagation(oneHot, X_train)

In [0]:
# error_list_sigmoid = []
# iter_list = []
# for i in range(400):
#         outputs = model_sigmoid.forwardPropagation(X_train)
#         c_entropy = crossEntropy(outputs[-1], oneHot)
# #         print(i, c_entropy)
#         error_list_sigmoid.append(c_entropy)
#         iter_list.append(i)
#         model_sigmoid.backPropagation(oneHot, X_train)

In [0]:
# error_list_tanh = []
# iter_list = []
# for i in range(400):
#         outputs = model_tanh.forwardPropagation(X_train)
#         c_entropy = crossEntropy(outputs[-1], oneHot)
# #         print(i, c_entropy)
#         error_list_tanh.append(c_entropy)
#         iter_list.append(i)
#         model_tanh.backPropagation(oneHot, X_train)

# Plotting Model Cost w.r.t Iterations 
## <font color = "blue">Models :  </font>


1.   **ReLU**
2.   **Sigmoid**
3.   **Tanh**






In [0]:
# fig, ax = plt.subplots(figsize=(12,6))
# ax.scatter(iter_list, error_list_relu, color="blue",marker = '.', s= 30, label = 'ReLU')
# ax.scatter(iter_list, error_list_sigmoid, color="black",marker = '.', s= 30, label = 'Sigmoid')
# ax.scatter(iter_list, error_list_tanh, color="red",marker = '.', s= 30, label = 'Tanh')
# plt.legend()
# plt.xlabel("Iterations")
# plt.ylabel("Cross Entropy Error")
# plt.show()

# Observation : 
##  <font color = "blue">ReLU and Tanh are performing exceptionally well with the current scenario. </font>
##  <font color = "blue">Surprisingly Tanh is performing better than ReLU in many runs. </font>
###  <font color = "blue">Though it is expected that ReLU will perform better than Tanh for most of the problems but it is not a guaranteed result. It is infered that the hyper parameters and the architecture favours Tanh</font>

In [0]:
# print("Minimum Error by ReLU : ", np.min(error_list_relu))
# print("Minimum Error by Tanh : ", np.min(error_list_tanh))
# print("Minimum Error by Sigmoid : ", np.min(error_list_sigmoid))

In [0]:
# # Relu accuracy
# val_output = model_relu.forwardPropagation(X_validate)
# predictions = np.argmax(val_output[-1], axis=1)
# y_validate = np.argmax(oneHotEncode(y_validate), axis = 1)
# print("RELU ACCURACY : ", (np.mean(predictions == y_validate))*100)

# # Sigmoid accuracy
# val_output = model_sigmoid.forwardPropagation(X_validate)
# predictions = np.argmax(val_output[-1], axis=1)
# y_validate = np.argmax(oneHotEncode(y_validate), axis = 1)
# print("SIGMOID ACCURACY : ", (np.mean(predictions == y_validate))*100)

# # Tanh accuracy
# val_output = model_tanh.forwardPropagation(X_validate)
# predictions = np.argmax(val_output[-1], axis=1)
# y_validate = np.argmax(oneHotEncode(y_validate), axis = 1)
# print("TANH ACCURACY : ", (np.mean(predictions == y_validate))*100)

In [0]:
# # Mini Batch 
# model_relu1 = neural_network(ReLU, ReLU_derivative, alpha_relu, input_dim, output_dim, hiddenLayers)
# batchSize = 32
# batchCount = int(X_train.shape[0] / batchSize)
# miniBatchError = []
# miniBatchIterList = []
# for iters in range(400):
#   startIndex = 0
#   endIndex = 32
#   miniBatchIterList.append(iters)
#   for batchNo in range(batchCount):
#     X_miniBatch = X_train[startIndex:endIndex]
#     outputs = model_relu1.forwardPropagation(X_miniBatch)
# #     print(outputs[-1])
#     y_miniBatch = oneHot[startIndex:endIndex]
#     if(batchNo == batchCount - 1):
#       c_entropy = crossEntropy(outputs[-1], y_miniBatch)
#       miniBatchError.append(c_entropy)
#     model_relu1.backPropagation(y_miniBatch, X_miniBatch)
#     startIndex = endIndex
#     endIndex += batchSize

In [0]:
# fig, ax = plt.subplots(figsize=(12,6))
# ax.scatter(miniBatchIterList, miniBatchError, color="blue",marker = '.', s= 30, label = 'ReLU')
# plt.legend()
# plt.show()

In [0]:
# val_output = model_relu1.forwardPropagation(X_validate)
# predictions = np.argmax(val_output[-1], axis=1)
# y_validate = np.argmax(oneHotEncode(y_validate), axis = 1)
# print("RELU ACCURACY : ", (np.mean(predictions == y_validate))*100)


In [0]:
def main(type):
#   if(type == "batch"):
#     train("batch")
#   else:
#     train("miniBatch")
  test_data = pd.read_csv("/content/gdrive/My Drive/SMAI_Assignments_colab/apparel-test.csv")
  test_normalized = StandardScaler().fit_transform(test_data)
  test(test_normalized)
    

In [0]:
# main("batch")

In [76]:
main("miniBatch")

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
